<a href="https://colab.research.google.com/github/Champioa00/LAB1/blob/master/LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required packages #
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
#from botocore.client import Config
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from fbprophet import Prophet
import statsmodels.api as sm
from  multiprocessing import Pool, cpu_count
import pandas as pd
import numpy as np
#import ibm_boto3
import itertools
import types
import math
import time

sns.set(rc={'figure.figsize':(11.7,8.27)})
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [21]:
df_data_1 = pd.read_csv('Zip_zhvi_uc_sfr_month.csv', encoding='ISO-8859-1')

In [23]:
df_data_1.head

<bound method NDFrame.head of        RegionID  SizeRank  RegionName  ...    11/30/2021    12/31/2021     1/31/2022
0         61639         0       10025  ...  2.600348e+06  2.563603e+06  2.729760e+06
1         84654         1       60657  ...  8.389180e+05  8.381373e+05  8.495258e+05
2         61637         2       10023  ...  5.143927e+06  5.115112e+06  5.111999e+06
3         91982         3       77494  ...  4.658952e+05  4.687325e+05  4.786334e+05
4         84616         4       60614  ...  1.131157e+06  1.127295e+06  1.141322e+06
...         ...       ...         ...  ...           ...           ...           ...
31985     87060     34430       66045  ...  2.756580e+05  2.719487e+05  2.724790e+05
31986     90986     34430       75599  ...  7.116641e+04  7.157647e+04  7.104333e+04
31987     93912     34430       82335  ...  2.461027e+05  2.621004e+05  2.633532e+05
31988     58112     35187         803  ...  1.623279e+05  1.699015e+05  1.750580e+05
31989     58117     35187         8

In [27]:
na_values = df_data_1.isna().sum()
print(na_values)

RegionID       0
SizeRank       0
RegionName     0
RegionType     0
StateName      0
              ..
9/30/2021     60
10/31/2021    44
11/30/2021    26
12/31/2021     0
1/31/2022      0
Length: 322, dtype: int64


In [26]:
metro_area = pd.read_csv('OD.csv')
metro_area.head()

,ZIP CODE,STATE,MSA No.,GPCI_W,GPCI_PE,GPCI_ME,County No.,MSA Name
0,210,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
1,211,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
2,212,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
3,213,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
4,214,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"


In [17]:
print(metro_area.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43508 entries, 0 to 43507
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ZIP CODE    43508 non-null  float64
 1   STATE       43508 non-null  object 
 2   MSA No.     43497 non-null  float64
 3   GPCI_W      43497 non-null  float64
 4   GPCI_PE     43497 non-null  float64
 5   GPCI_ME     43497 non-null  float64
 6   County No.  43496 non-null  float64
 7   MSA Name    43497 non-null  object 
dtypes: float64(6), object(2)
memory usage: 2.7+ MB
None


In [16]:
## Clean up Numeric Columns ## 
## Select all the numeric columns ## 
MA_num_columns = ['ZIP CODE']

## Remove $, ",", and replace "--" with "0" ## 
for i in MA_num_columns:
    metro_area[i] = metro_area[i].astype(float)

In [19]:
print(df_data_1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4891 entries, 0 to 14
Columns: 322 entries, RegionID to 1/31/2022
dtypes: float64(313), int64(3), object(6)
memory usage: 12.1+ MB
None


In [10]:
# Keep data with missing metro name. 
missing_metro = df_data_1[df_data_1['Metro'].isna()]
missing_index = missing_metro.index

# Drop observation without Metro data. These observations will then be reattached
df_metro_complete = df_data_1.drop(index=missing_index)

# Keep zip code and msa name from the metro_area data set
metro_area_simp = metro_area[['ZIP CODE', 'MSA Name']]

# Merge missing metro with metro_area_simp 
metro_complete = missing_metro.merge(metro_area_simp, how='left', left_on='RegionName', right_on='ZIP CODE')

In [11]:
# Check to see that there are no NaN variables in the new column. 
print(metro_complete['MSA Name'].isna().sum())

# There are no NaN values now. We'll now replace the Metro column with the MSA Name column, drop the overlapping columns, and 
# reattach the observations to the df_data_1 data set. 
metro_complete['Metro'] = metro_complete['MSA Name']
metro_complete1 = metro_complete.iloc[:, 0:280]

# Reattach data to original data. 
df_data_1 = df_metro_complete.append(metro_complete1)
print(df_data_1.shape)

0
(4891, 322)


In [18]:
# Obtain column names of year-month combinations. 
year_month = df_data_1.iloc[:, 7:].columns.values

df_data_2 = df_data_1




TypeError: ignored

In [ ]:
coachdf.head()

In [14]:
df_data_2.describe()

,RegionID,SizeRank,RegionName,1/31/1996,2/29/1996,3/31/1996,4/30/1996,5/31/1996,6/30/1996,7/31/1996,8/31/1996,9/30/1996,10/31/1996,11/30/1996,12/31/1996,1/31/1997,2/28/1997,3/31/1997,4/30/1997,5/31/1997,6/30/1997,7/31/1997,8/31/1997,9/30/1997,10/31/1997,11/30/1997,12/31/1997,1/31/1998,2/28/1998,3/31/1998,4/30/1998,5/31/1998,6/30/1998,7/31/1998,8/31/1998,9/30/1998,10/31/1998,11/30/1998,12/31/1998,1/31/1999,...,10/31/2018,11/30/2018,12/31/2018,1/31/2019,2/28/2019,3/31/2019,4/30/2019,5/31/2019,6/30/2019,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,7/31/2020,8/31/2020,9/30/2020,10/31/2020,11/30/2020,12/31/2020,1/31/2021,2/28/2021,3/31/2021,4/30/2021,5/31/2021,6/30/2021,7/31/2021,8/31/2021,9/30/2021,10/31/2021,11/30/2021,12/31/2021,1/31/2022
count,4891.000000,4891.000000,4891.000000,3.991000e+03,3.996000e+03,3.997000e+03,3.997000e+03,4.012000e+03,4.014000e+03,4.015000e+03,4.020000e+03,4.024000e+03,4.025000e+03,4.026000e+03,4.030000e+03,4.078000e+03,4.088000e+03,4.094000e+03,4.097000e+03,4.101000e+03,4.104000e+03,4.107000e+03,4.113000e+03,4.125000e+03,4.135000e+03,4.144000e+03,4.144000e+03,4.157000e+03,4.159000e+03,4.159000e+03,4.195000e+03,4.212000e+03,4.238000e+03,4.238000e+03,4.243000e+03,4.246000e+03,4.249000e+03,4.249000e+03,4.249000e+03,4.261000e+03,...,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.868000e+03,4.872000e+03,4.872000e+03,4.872000e+03,4.872000e+03,4.873000e+03,4.873000e+03,4.873000e+03,4.873000e+03,4.873000e+03,4.873000e+03,4.874000e+03,4.875000e+03,4.875000e+03,4.875000e+03,4.875000e+03,4.875000e+03,4.875000e+03,4.875000e+03
mean,84275.123288,2456.670006,53902.088734,1.515716e+05,1.529418e+05,1.528506e+05,1.532388e+05,1.545984e+05,1.550101e+05,1.553836e+05,1.560320e+05,1.563300e+05,1.568470e+05,1.572448e+05,1.576411e+05,1.575664e+05,1.576165e+05,1.580957e+05,1.586382e+05,1.596196e+05,1.603769e+05,1.612207e+05,1.618041e+05,1.621270e+05,1.622298e+05,1.630068e+05,1.637010e+05,1.649293e+05,1.654510e+05,1.662422e+05,1.667689e+05,1.680043e+05,1.689140e+05,1.700052e+05,1.710178e+05,1.725135e+05,1.732939e+05,1.741752e+05,1.748889e+05,1.746129e+05,...,4.007355e+05,4.012515e+05,4.016765e+05,4.017700e+05,4.030124e+05,4.046060e+05,4.059544e+05,4.078462e+05,4.083428e+05,4.057702e+05,4.057148e+05,4.060343e+05,4.062117e+05,4.069173e+05,4.090590e+05,4.119769e+05,4.146991e+05,4.180269e+05,4.201868e+05,4.180860e+05,4.198867e+05,4.225577e+05,4.260330e+05,4.300346e+05,4.344319e+05,4.375911e+05,4.418355e+05,4.458393e+05,4.524976e+05,4.587852e+05,4.675342e+05,4.789575e+05,4.884832e+05,4.962626e+05,5.008148e+05,5.051410e+05,5.097304e+05,5.150101e+05,5.218541e+05,5.303707e+05
std,37003.807435,1422.653427,29709.964819,9.782399e+04,1.291423e+05,1.262192e+05,1.296132e+05,1.429832e+05,1.463583e+05,1.488828e+05,1.535616e+05,1.548195e+05,1.577941e+05,1.610714e+05,1.617173e+05,1.594685e+05,1.550240e+05,1.549289e+05,1.535183e+05,1.553049e+05,1.598308e+05,1.662828e+05,1.706031e+05,1.659599e+05,1.687156e+05,1.725388e+05,1.720780e+05,1.768429e+05,1.781251e+05,1.800009e+05,1.868512e+05,1.941183e+05,1.993471e+05,2.074613e+05,2.019145e+05,2.081817e+05,2.088792e+05,2.125261e+05,2.173305e+05,2.086426e+05,...,4.990350e+05,4.961646e+05,4.943336e+05,4.884611e+05,4.877838e+05,4.896722e+05,4.864542e+05,4.938722e+05,4.956787e+05,4.878986e+05,4.840485e+05,4.805829e+05,4.769713e+05,4.753799e+05,4.792857e+05,4.869602e+05,4.861511e+05,4.851765e+05,4.908955e+05,4.849937e+05,4.867519e+05,4.821467e+05,4.812211e+05,4.830803e+05,4.799388e+05,4.772458e+05,4.809637e+05,4.755305e+05,4.829363e+05,4.861645e+05,4.968056e+05,5.070884e+05,5.106879e+05,5.222681e+05,5.230124e+05,5.271846e+05,5.260699e+05,5.277023e+05,5.329724e+05,5.390663e+05
min,58004.000000,0.000000,602.000